In [ ]:
from argparse import Action
from tkinter import E
from turtle import shape
from aiohttp import request
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


import numpy as np
import random
import os
import win32api, win32con, win32gui
from PIL import ImageGrab
import time
import requests
import PIL
from PIL import Image
import mss
from matplotlib import pyplot as plt
import re
import cv2

In [ ]:
global holding
holding = False

def move_cursor(x, y): # jde od 1 do 1920
    nx = int(x*65535/win32api.GetSystemMetrics(0))
    ny = int(y*65535/win32api.GetSystemMetrics(1))
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE|win32con.MOUSEEVENTF_MOVE,nx,ny)
    return (x,y)

def move_in_sreen(x,y):
    x = int(x)
    y = int(y)
    if x < 240:
        x = 241
    if x > 1440:
        x = 1439
    return move_cursor(x,y)

def click(x,y):
    global holding
    if holding:
        holding = False
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y)
    else:
        move_cursor(x, y)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    return (x,y)


def click_in_sreen(x,y):
    x = int(x)
    y = int(y)
    if x < 240:
        x = 241
    if x > 1440:
        x = 1439
    return click(x,y)

def hold(x, y):
    global holding
    if holding:
        move_cursor(x, y)
    else:
        move_cursor(x, y)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y)

    holding = True
    return (x, y)

def hold_in_sreen(x,y):
    x = int(x)
    y = int(y)
    if x < 240:
        x = 241
    if x > 1440:
        x = 1439
    return hold(x,y)


def take_screanshot(): #0.05 sec
    normalized_alfa_of_image = []
    image = ImageGrab.grab(bbox=(239,0,1679,1079))

    image.thumbnail((108, 81), PIL.Image.ANTIALIAS)#36/27 z 1440/1080
    image = image.convert('L')
    #image.show()
    for x in range(image.height):
        row = []
        for y in range(image.width):
            color = image.getpixel((y, x))
            color /= 255
            row.append(color)
        normalized_alfa_of_image.append(row)

    return np.reshape(np.array(normalized_alfa_of_image), (81,108,1))



def take_screanshot_faster():  #30fps
    with mss.mss() as sct:
        monitor = {"top": 0, "left": 239, "width": 1440, "height": 1079}
        image = sct.grab(monitor)
        img = Image.frombytes("RGB", image.size, image.bgra, "raw", "BGRX")
        img.thumbnail((108,81), PIL.Image.ANTIALIAS)#36/27 z 1440/1080
        img = img.convert('L')
        return np.reshape(np.array(img), (81,108,1))

def take_screanshot_faster_squished():
    with mss.mss() as sct:
        monitor = {"top": 0, "left": 239, "width": 1440, "height": 1079}
        image = sct.grab(monitor)
        img = Image.frombytes("RGB", image.size, image.bgra, "raw", "BGRX")
        img.thumbnail((36, 27), PIL.Image.ANTIALIAS)#36/27 z 1440/1080
        img = img.convert('L')
        img = np.reshape(np.array(img), (27,36,1))
        img = img /255
        return img

def gamedata():
    ready_to_deliver = False

    while not ready_to_deliver:
        try:
            data = requests.get("http://127.0.0.1:24050/json").json()
            ready_to_deliver = True
        except Exception as e:
            print(e)
            time.sleep(0.1)
            
    return data

In [ ]:
class WindowMgr:
    """Encapsulates some calls to the winapi for window management"""

    def __init__ (self):
        """Constructor"""
        self._handle = None

    def find_window(self, class_name, window_name=None):
        """find a window by its class_name"""
        self._handle = win32gui.FindWindow(class_name, window_name)


    def maximaze (self):
        win32gui.ShowWindow(self._handle, win32con.SW_MAXIMIZE)

    def _window_enum_callback(self, hwnd, wildcard):
        """Pass to win32gui.EnumWindows() to check all the opened windows"""
        if re.match(wildcard, str(win32gui.GetWindowText(hwnd))) is not None:
            self._handle = hwnd

    def find_window_wildcard(self, wildcard):
        """find a window whose title matches the wildcard regex"""
        self._handle = None
        win32gui.EnumWindows(self._window_enum_callback, wildcard)

In [ ]:
class ozu_driver(Env):
    def __init__(self):
        #setup the game
        w = WindowMgr()
        w.find_window_wildcard("osu")
        w.maximaze()          #nefunguje

        self.action_space = Box(-1,1, shape=(13,))
        #                                                                             #float32 for real numbers                                      #unit8 for natural numbers
        self.observation_space = Dict({'cursor_pos': Box(low=-1, high=1, shape=(2,1), dtype=np.float32),"screan":Box(low=0, high=1, shape=(27,36,1), dtype=np.uint8)})
        self.start = time.time()
        global holding
        holding = False
        self.state = {
            'cursor_pos': np.array( [[(((win32api.GetCursorPos()[0]-239)/1440)*2 -1)], [(win32api.GetCursorPos()[1]/1080)*2 -1]] ),
            "screan":take_screanshot_faster_squished()}
        self.map_lenhgt = int(gamedata()["menu"]["bm"]["time"]["full"])/1000
        self.hit0 = 0
        self.hit50 = 0
        self.hit100 = 0
        self.hit300 = 0
        self.cursor = win32api.GetCursorPos()
        self.objects =  int(gamedata()["menu"]["bm"]["stats"]["maxCombo"])


        global screans_of_reward                                                            #debug tool
        screans_of_reward = []
        global rews
        rews = []


    def do_action(self,action):   #pohne nebo klikne a pohne nebo drží a pohne
        move_by = 1/30
        resolution = 1080
        pixel_movement = move_by*resolution

        if action == 0:
            self.cursor = move_in_sreen(self.cursor[0],self.cursor[1]+pixel_movement) #dolu
        if action == 1:
            self.cursor = move_in_sreen(self.cursor[0]+pixel_movement,self.cursor[1]) #doleva
        if action == 2:
            self.cursor = move_in_sreen(self.cursor[0],self.cursor[1]-pixel_movement) #nahoru
        if action == 3:
            self.cursor = move_in_sreen(self.cursor[0]-pixel_movement,self.cursor[1]) #doprava

        if action == 4:
            self.cursor = click_in_sreen(self.cursor[0],self.cursor[1]+pixel_movement) #click dolu
        if action == 5:
            self.cursor = click_in_sreen(self.cursor[0]+pixel_movement,self.cursor[1]) #click doleva
        if action == 6:
            self.cursor = click_in_sreen(self.cursor[0],self.cursor[1]-pixel_movement) #click nahoru
        if action == 7:
            self.cursor = click_in_sreen(self.cursor[0]-pixel_movement,self.cursor[1]) #click doprava

        if action == 8:
            self.cursor = hold_in_sreen(self.cursor[0],self.cursor[1]+pixel_movement) #hold dolu
        if action == 9:
            self.cursor = hold_in_sreen(self.cursor[0]+pixel_movement,self.cursor[1]) #hold doleva
        if action == 10:
            self.cursor = hold_in_sreen(self.cursor[0],self.cursor[1]-pixel_movement) #hold nahoru
        if action == 11:
            self.cursor = hold_in_sreen(self.cursor[0]-pixel_movement,self.cursor[1]) #hold doprava
        
        if action == 12:
            pass

    def step(self, action):
        self.cursor = win32api.GetCursorPos()
        action = action.argmax()
        self.do_action(action)
        """
        if action[0] > 0.5:
            click(int(action[2]*1440 - 1 + 240),int(action[3]*1080 -1))     # takhle se to vubec nema hýbat je t dojebany

        elif action[1] > 0.5:
            hold(int(action[2]*1440 - 1 + 240), int(action[3]*1080 -1))

        else:
            move_cursor(int(action[2]*1440 + 240),int(action[3]*1080))
        
        """
        """
        self.cursor = win32api.GetCursorPos()
        max_mvement_per_frame = 5                            # čím větší tím menší movement per frame
        if (int(self.cursor[0] + ((action[3]*1919)/(4*max_mvement_per_frame))) >= 239)  and ((int(self.cursor[0] + (((action[3]*1919)/(4*max_mvement_per_frame))))) <= 1679):      #jestli by se kurzor měl pohnout z hratelného pole tak se pohne do opačného směru
            if action[0] > 0:
                click(int(self.cursor[0] + ((action[2]*1079)/(3*max_mvement_per_frame))), int(self.cursor[1] + ((action[3]*1919)/(4*max_mvement_per_frame))))     # pohyb relativni ke kurzoru 

            elif action[1] > 0:
                hold(int(self.cursor[0] + ((action[2]*1079)/(3*max_mvement_per_frame))), int(self.cursor[1] + ((action[3]*1919)/(4*max_mvement_per_frame))))

            else:
                move_cursor(int(self.cursor[0] + ((action[2]*1079)/(3*max_mvement_per_frame))), int(self.cursor[1] + ((action[3]*1919)/(4*max_mvement_per_frame))))
        else:
            move_cursor(int(self.cursor[0] - ((action[2]*1079)/(3*max_mvement_per_frame))), int(self.cursor[1] - ((action[3]*1919)/(4*max_mvement_per_frame))))

        """


        frame_stats = gamedata()["gameplay"]["hits"]
        
        reward = (
        ((frame_stats["300"] - self.hit300)*300)+ 
        ((frame_stats["100"] - self.hit100)*100)+ 
        ((frame_stats["50"] - self.hit50)*50)+
        ((frame_stats["0"] - self.hit0)*-300)
        )/self.objects

        self.hit300 = frame_stats["300"]
        self.hit100 = frame_stats["100"]
        self.hit50 = frame_stats["50"]
        self.hit0 = frame_stats["0"]

        info = {}

        if time.time() - self.start >= self.map_lenhgt:
            done = True
            acc = gamedata()["gameplay"]["accuracy"]                        #debug tool (can be deleted)
            print(f"done: {done}\nacc: {acc}")                                                                      #debug tool (can be deleted)
            time.sleep(10)
            move_cursor(1920/2,1080/2)  #kurzor doprostřed obrazovky aby se skipnul end
            time.sleep(5)

        else:
            done = False
        
        
        #cur_screan = take_screanshot()
        cur_screan = take_screanshot_faster_squished()
        self.state = {
            'cursor_pos': np.array( [[(((win32api.GetCursorPos()[0]-239)/1440)*2 -1)], [(win32api.GetCursorPos()[1]/1080)*2 -1]] ),
            "screan":cur_screan}
        







        """
        print(self.state["cursor_pos_x"],self.state["cursor_pos_y"])
        print(action)
        print(reward)
        
            

        """ 
        #print (self.cursor, (self.map_lenhgt - (time.time() - self.start)))
        global screans_of_reward                                                            #debug tool (can be deleted)
        if reward != 0:
            if abs(reward) > 2:
                rews.append(reward)                                                                  #debug tool (can be deleted)
                screans_of_reward.append(self.state["screan"])
            #print((self.state["cursor_pos_x"], self.state["cursor_pos_y"]), reward*self.objects, (self.map_lenhgt - (time.time() - self.start)), sum(sum(self.state["screan"])))
            #plt.imshow(self.state["screan"])
            #print(self.state["screan"])
        
        
        return self.state , reward, done, info


    def render(self):
        pass

    def reset(self):
        print("reseting")
        global holding
        holding = False
        time.sleep(10)
        move_cursor(50, 1020)  # go back from done run to song select
        time.sleep(0.3)
        click(50, 1020)
        time.sleep(0.1)
        click(50, 1020)
        time.sleep(3)
        #click(600,1000)    #randomize songs
        time.sleep(5)
        click(1500, 500)    #chose song
        time.sleep(1)
        self.start = time.time()
        time.sleep(2)
        move_cursor(1920/2,1080/2)  #kurzor doprostřed obrazovky



        self.cursor = (1920/2,1080/2)
        self.map_lenhgt = gamedata()["menu"]["bm"]["time"]["full"]/1000 - gamedata()["menu"]["bm"]["time"]["firstObj"]/1000 + 4
        self.objects = int(gamedata()["menu"]["bm"]["stats"]["maxCombo"])
        
        self.hit300 = 0
        self.hit100 = 0
        self.hit50 = 0
        self.hit0 = 0


        return {
            'cursor_pos': np.array( [[(((win32api.GetCursorPos()[0]-239)/1440)*2 -1)], [(win32api.GetCursorPos()[1]/1080)*2 -1]] ),
            "screan":take_screanshot_faster_squished()}

In [ ]:

#env = ozu_driver()

In [ ]:
#state = env.step([0,0,0,0])
#plt.imshow(cv2.cvtColor(state[0], cv2.COLOR_BGR2RGB))
#print(state[0].shape)


In [ ]:
#from stable_baselines3.common import env_checker
#env_checker.check_env(env)

In [ ]:
""""
time.sleep(1)
print("start")
start = time.time()


for i in range(100):
    
    a = take_screanshot_faster()
    



print(time.time() - start)
"""

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
dir_id = time.time()
minutes = 60
info = f"not_random_normalised_mouse_normalised_new_skin_36_27__out_12"

CHECKPOINT_DIR = f'./train/{minutes}_minutes_{info}_{dir_id}'
LOG_DIR = f'./logs/{minutes}_minutes_{info}_{dir_id}'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
env = ozu_driver()

framerate = 20
#model = PPO("MultiInputPolicy", env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001)
model = PPO.load(r"train\30_minutes_not_random_normalised_mouse_normalised_new_skin_36_27__out_12_1677415207.0267646\best_model_20000.zip", env=env)

In [ ]:

model.learn(total_timesteps=(framerate*60*minutes), callback=callback)

In [ ]:
global screans_of_reward
a = 1

#print(screans_of_reward)
plt.imshow(screans_of_reward[a])
print(screans_of_reward[a].shape)
for i in rews:
    print (i)
#plt.imshow(take_screanshot())

In [ ]:
a = time.time()
print(np.array( [[(((win32api.GetCursorPos()[0]-239)/1440)*2 -1)], [(win32api.GetCursorPos()[1]/1080)*2 -1]] ))
gamedata()
take_screanshot_faster_squished()
print(time.time() - a)
np.array( [[(((win32api.GetCursorPos()[0]-239)/1440)*2 -1)], [(win32api.GetCursorPos()[1]/1080)*2 -1]] )